## 04 - Generate and interferogram using SNAP

### Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

### <a name="objective">Objective 




### <a name="data">Data

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

All data products are distributed in the SENTINEL Standard Archive Format for Europe (SAFE) format.

Data products are available in single polarisation (VV or HH) for Wave mode and dual polarisation (VV+VH or HH+HV) and single polarisation (HH or VV) for SM, IW and EW modes.

In [1]:
input_identifiers = ('S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A',
                     'S1B_IW_SLC__1SDV_20160930T163111_20160930T163139_002301_003E3A_4516')

**Data path**

This path defines where the data is staged-in. 

In [6]:
data_path = "/workspace/data"

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [7]:
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
from datetime import datetime

import cioppy

import gdal
import osr

from shapely.wkt import loads
from shapely.geometry import box

import lxml.etree as etree
import numpy as np

#### Read the products

In [8]:
s1meta = "manifest.safe"

for index, s1path in enumerate(input_identifiers):

    s1prd= "%s/%s/%s.SAFE/%s" % (data_path, s1path, s1path, s1meta)
    reader = ProductIO.getProductReader("SENTINEL-1")
    
    if index == 0:
        slave = reader.readProductNodes(s1prd, None)
        slave_date = parser.parse(slave.getStartTime().toString()).isoformat()[:19]
    else:
        master = reader.readProductNodes(s1prd, None)
        master_date = parser.parse(slave.getStartTime().toString()).isoformat()[:19]

### Apply orbit file

In [9]:
operator = 'Apply-Orbit-File'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('orbitType', 'Sentinel Precise (Auto Download)')
('polyDegree', '3')
('continueOnFail', 'false')


In [10]:

parameters = HashMap()

parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
parameters.put('polyDegree', '3')
parameters.put('continueOnFail', 'false')


master_orbit = GPF.createProduct(operator,
                                 parameters, 
                                 master)
    
slave_orbit = GPF.createProduct(operator,
                                 parameters, 
                                 slave)



### TOPSAR split

In [11]:
operator = 'TOPSAR-Split'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('subswath', None)
('selectedPolarisations', None)
('firstBurstIndex', '1')
('lastBurstIndex', '9999')
('wktAoi', None)


In [12]:
slave_split_prds = []
master_split_prds = []

for subswath in ['IW1', 'IW2', 'IW3']:  
    
    parameters = HashMap()

    parameters.put('subswath', subswath)
    parameters.put('selectedPolarisations', 'VV')
    parameters.put('firstBurstIndex', '1')
    parameters.put('lastBurstIndex', '9999')

    master_split_prds.append(GPF.createProduct(operator,
                           parameters, 
                           master_orbit))   
    
    slave_split_prds.append(GPF.createProduct(operator,
                           parameters, 
                           slave_orbit))   
    

### Back-Geocoding

In [13]:
operator = 'Back-Geocoding'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('demName', 'SRTM 3Sec')
('demResamplingMethod', 'BICUBIC_INTERPOLATION')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('resamplingType', 'BISINC_5_POINT_INTERPOLATION')
('maskOutAreaWithoutElevation', 'true')
('outputRangeAzimuthOffset', 'false')
('outputDerampDemodPhase', 'false')
('disableReramp', 'false')


In [14]:
backgeo_prds = []

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    

    parameters = HashMap()

    for param in op_params:
        parameters.put(param.getName(), param.getDefaultValue())


    backgeo_prds.append(GPF.createProduct(operator,
                           parameters, 
                           [master_split_prds[index], 
                            slave_split_prds[index]
                           ]))

### Interferogram

In [15]:
operator = 'Interferogram'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

parameters = HashMap()

for param in op_params:
    print(param.getName(), param.getDefaultValue())
    parameters.put(param.getName(), param.getDefaultValue())

('subtractFlatEarthPhase', 'true')
('srpPolynomialDegree', '5')
('srpNumberPoints', '501')
('orbitDegree', '3')
('includeCoherence', 'true')
('cohWinAz', '10')
('cohWinRg', '10')
('squarePixel', 'true')
('subtractTopographicPhase', 'false')
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('externalDEMApplyEGM', 'true')
('tileExtensionPercent', '100')
('outputElevation', 'false')
('outputLatLon', 'false')


In [16]:
interferogram_prds = []

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    
    parameters = HashMap()

    for param in op_params:
        parameters.put(param.getName(), param.getDefaultValue())

    interferogram_prds.append(GPF.createProduct(operator,
                           parameters, 
                           backgeo_prds[index]))

#### TOPSAR Deburst

In [17]:
operator = 'TOPSAR-Deburst'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())


('selectedPolarisations', None)


In [18]:
deburst_prds = []

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
 
    parameters = HashMap()
    parameters.put('selectedPolarisations', 'VV')

    deburst_prds.append(GPF.createProduct(operator,
                           parameters, 
                           interferogram_prds[index]))

#### TOPSAR Merge

In [19]:
operator = 'TOPSAR-Merge'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('selectedPolarisations', None)


In [20]:
parameters = HashMap()
parameters.put('selectedPolarisations', 'VV')

tops_merge = GPF.createProduct(operator,
                           parameters, 
                           deburst_prds)


#### TopoPhaseRemoval

In [21]:
operator = 'TopoPhaseRemoval'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('orbitDegree', '3')
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('tileExtensionPercent', '100')
('outputTopoPhaseBand', 'false')
('outputElevationBand', 'false')
('outputLatLonBands', 'false')


In [22]:
parameters = HashMap()

for param in op_params:
    parameters.put(param.getName(), param.getDefaultValue())


topo_phase = GPF.createProduct(operator,
                  parameters, 
                  tops_merge)

#### GoldsteinPhaseFiltering

In [23]:
operator = 'GoldsteinPhaseFiltering'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('alpha', '1.0')
('FFTSizeString', '64')
('windowSizeString', '3')
('useCoherenceMask', 'false')
('coherenceThreshold', '0.2')


In [24]:
parameters = HashMap()

for param in op_params:
    parameters.put(param.getName(), param.getDefaultValue())


goldstein = GPF.createProduct(operator,
                  parameters, 
                  topo_phase)

#### Terrain correction

In [25]:
operator = 'Terrain-Correction'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('sourceBandNames', None)
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('externalDEMApplyEGM', 'true')
('demResamplingMethod', 'BILINEAR_INTERPOLATION')
('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
('pixelSpacingInMeter', '0')
('pixelSpacingInDegree', '0')
('mapProjection', 'WGS84(DD)')
('alignToStandardGrid', 'false')
('standardGridOriginX', '0')
('standardGridOriginY', '0')
('nodataValueAtSea', 'true')
('saveDEM', 'false')
('saveLatLon', 'false')
('saveIncidenceAngleFromEllipsoid', 'false')
('saveLocalIncidenceAngle', 'false')
('saveProjectedLocalIncidenceAngle', 'false')
('saveSelectedSourceBand', 'true')
('outputComplex', 'false')
('applyRadiometricNormalization', 'false')
('saveSigmaNought', 'false')
('saveGammaNought', 'false')
('saveBetaNought', 'false')
('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')
('incidenceAngleForGamma0', 'Use projected local incidence angle from DEM')
('auxFile', 'Latest Auxiliary Fil

In [26]:
parameters = HashMap()

for param in op_params:
    parameters.put(param.getName(), param.getDefaultValue())


terrain = GPF.createProduct(operator,
                  parameters, 
                  goldstein)

In [ ]:
ProductIO.writeProduct(terrain, 
                       'output_name' + '.tif',
                       'GeoTIFF-BigTIFF')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.